In [3]:
import pandas as pd
import numpy as np
import copy
import string
import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
nltk.download('punkt')
from scipy.optimize import minimize,curve_fit
import matplotlib.pyplot as plt
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to /home/sachin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
train_df = pd.read_csv('train_dataset.csv')

In [5]:
test_df = pd.read_csv('test_dataset.csv')

In [6]:
comments='Comment'

In [7]:
from nltk.corpus import stopwords
lemmatizer = WordNetLemmatizer()

In [8]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/sachin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
stops = set(stopwords.words('english'))

In [10]:
train_sentences = []
train_list=[]

for comment in train_df['Comment']:
    sentences = sent_tokenize(comment)
    train_list.append(sentences)
for data in train_list:
    for sent in data:
        train_sentences.append(sent)
test_sentences = []
test_list=[]
for comment in test_df['Comment']:
    sentences = sent_tokenize(comment)
    test_list.append(sentences)
for data in test_list:
    for sent in data:
        test_sentences.append(sent)

In [11]:
translator = str.maketrans('', '', string.punctuation)
train_sentences_temp=[]
for sent in train_sentences:
    sentence_without_commas = sent
    sentence_without_commas = sentence_without_commas.replace(',', '')
    sentence_without_commas = sentence_without_commas.replace(',', '')
    sentence_without_commas = sentence_without_commas.replace('.', '')
    sentence_without_commas = sentence_without_commas.replace('!', '')
    sentence_without_commas = sentence_without_commas.replace('?', '')
    sentence_without_commas = sentence_without_commas.replace('-', '')
    sentence_without_commas = sentence_without_commas.replace('"', '')
    sentence_without_commas = sentence_without_commas.replace(')', '')
    sentence_without_commas = sentence_without_commas.replace('(', '')
    sentence_without_commas = sentence_without_commas.replace(']', '')
    sentence_without_commas = sentence_without_commas.replace('[', '')
    sentence_without_commas = sentence_without_commas.replace("'", '')
    sentence_without_commas = sentence_without_commas.replace('{', '')
    sentence_without_commas = sentence_without_commas.replace('}', '')
    sentence_without_commas = sentence_without_commas.replace('%', '')
    sentence_without_commas = sentence_without_commas.replace('$', '')
    train_sentences_temp.append(sentence_without_commas)
train_sentences=train_sentences_temp
test_sentences_temp=[]
for sent in test_sentences:
    sentence_without_commas = sent
    sentence_without_commas = sentence_without_commas.replace(',', '')
    sentence_without_commas = sentence_without_commas.replace(',', '')
    sentence_without_commas = sentence_without_commas.replace('.', '')
    sentence_without_commas = sentence_without_commas.replace('!', '')
    sentence_without_commas = sentence_without_commas.replace('?', '')
    sentence_without_commas = sentence_without_commas.replace('-', '')
    sentence_without_commas = sentence_without_commas.replace('"', '')
    sentence_without_commas = sentence_without_commas.replace(')', '')
    sentence_without_commas = sentence_without_commas.replace('(', '')
    sentence_without_commas = sentence_without_commas.replace(']', '')
    sentence_without_commas = sentence_without_commas.replace('[', '')
    sentence_without_commas = sentence_without_commas.replace("'", '')
    sentence_without_commas = sentence_without_commas.replace('{', '')
    sentence_without_commas = sentence_without_commas.replace('}', '')
    sentence_without_commas = sentence_without_commas.replace('%', '')
    sentence_without_commas = sentence_without_commas.replace('$', '')
    test_sentences_temp.append(sentence_without_commas)
test_sentences=test_sentences_temp


In [12]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/sachin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [13]:
train_unigram = []
for sent in train_sentences:
    words = word_tokenize(sent)
    proc_words = []
    for word in words:
        if(word.isalpha() and word not in stops): proc_words.append(lemmatizer.lemmatize(word.lower()))
    if len(proc_words)!=0:
        train_unigram.append(proc_words)
test_unigram = []
for sent in test_sentences:
    words = word_tokenize(sent)
    proc_words = []
    for word in words:
        if(word.isalpha() and word not in stops): proc_words.append(lemmatizer.lemmatize(word.lower()))
    if len(proc_words)!=0:
        test_unigram.append(proc_words)

train_bigram = copy.deepcopy(train_unigram)
test_bigram = copy.deepcopy(test_unigram)
train_trigram = copy.deepcopy(train_unigram)
test_trigram = copy.deepcopy(test_unigram)
train_quadgram = copy.deepcopy(train_unigram)
test_quadgram = copy.deepcopy(test_unigram)

In [14]:
for sent in train_bigram:
    sent.append('</s>')
    sent.insert(0, '<s>')
for sent in test_bigram:
    sent.append('</s>')
    sent.insert(0, '<s>')

In [15]:
for sent in train_trigram:
    sent.append('</s>')
    sent.insert(0, '<s>')
    sent.append('</s>')
    sent.insert(0, '<s>')
for sent in test_trigram:
    sent.append('</s>')
    sent.insert(0, '<s>')
    sent.append('</s>')
    sent.insert(0, '<s>')

In [16]:
for sent in train_quadgram:
    sent.append('</s>')
    sent.insert(0, '<s>')
    sent.append('</s>')
    sent.insert(0, '<s>')
    sent.append('</s>')
    sent.insert(0, '<s>')
for sent in test_quadgram:
    sent.append('</s>')
    sent.insert(0, '<s>')
    sent.append('</s>')
    sent.insert(0, '<s>')
    sent.append('</s>')
    sent.insert(0, '<s>')

In [39]:
import numpy as np

start_sent = "<s>"
end_sent = "</s>"


class unigram_model:
    def __init__(self, sentences):
        self.unigram_frequencies = dict()
        self.good_turing_count = dict()
        self.vocabulary = set()
        self.corpus_size = 0
        for sentence in sentences:
            for word in sentence:
                self.unigram_frequencies[word] = (
                    self.unigram_frequencies.get(word, 0) + 1
                )
                if word != start_sent or word != end_sent:
                    self.corpus_size += 1
                if word not in self.vocabulary:
                    self.vocabulary.add(word)
        self.vocab_size = (
            len(self.unigram_frequencies) - 2
        )  # Not including start and end of sentence in vocabulary

    def calculate_probability(self, word):
        if word not in self.vocabulary:
            return 0
        return float(self.unigram_frequencies[word]) / float(self.corpus_size)

    def calculate_perplexity(self, word):
        prob_word = self.calculate_probability(word)
        if prob_word == 0:
            return 10 ** (7)
        perplexity = np.log2(float(1 / prob_word)) * (float(1 / self.corpus_size))
        return perplexity

    def calculate_probability_smooth(self, word, k):
        return float(self.unigram_frequencies[word] + float(k)) / (
            float(self.corpus_size) + k * float(self.vocab_size)
        )

    def calculate_new_word_count(self,diction:dict):
        # freq_to_words_dict : {Key:Number of words occured that many times, Value:List of words that occured that many times}
        freq_to_words_dict = dict()

        for word in diction.keys():
            if(diction[word] in freq_to_words_dict.keys()):
                freq_to_words_dict[diction[word]] += [word]
            else:
                freq_to_words_dict[diction[word]] = [word]
        # freq_to_words_dict[0] = ["<unk>"]
        # new_word_count : {Key:Word, Value:New count of word according to good turing}
        x_values = list(freq_to_words_dict.keys())
        x_values_2=[]
        y_values = []
        for i in x_values:
            if(i+1 not in freq_to_words_dict.keys()):
                x_values_2.append(i)
                y_values.append(len(freq_to_words_dict[i]))
        x_values = np.array(x_values_2)
        y_values = np.array(y_values)
        x_values_3=x_values.argsort()[:5]
        x_values=x_values[x_values_3]
        y_values=y_values[x_values_3]
        # print(x_values)

        def power_law(x, C, alpha):
            return C * np.power(x, -alpha)

        params, _ = curve_fit(power_law, x_values, y_values)

        new_word_count = dict()
        keys = freq_to_words_dict.keys()
        for i in freq_to_words_dict.keys():
            for word in freq_to_words_dict[i]:
                if i + 1 in freq_to_words_dict.keys():
                    new_word_count[word] = (i + 1) * (
                        len(freq_to_words_dict[i + 1]) / len(freq_to_words_dict[i])
                    )
                else:
                    new_word_count[word] = (i + 1) * (
                        power_law(float(i + 1), params[0], params[1])
                        / power_law(float(i), params[0], params[1])
                    )
                # new_word_count[word] = (
                #     (self.unigram_frequencies[word] + 1) * power_law / keys[i]
                # )
        # Keeping the word count same for the most frequent words
        # Calculating the probability of <unk>, <unk> is which has not appeared in the corpus
        new_word_count["<unk>"] = len(freq_to_words_dict[1]) / self.corpus_size
        self.good_turing_count = new_word_count

    def calculate_probability_good_smooth(self, word):
      new_word_count = self.good_turing_count
      if word not in new_word_count:
          return new_word_count["<unk>"]
      return new_word_count[word] / self.corpus_size


class bigram_model(unigram_model):
    def __init__(self, sentences):
        unigram_model.__init__(self, sentences)
        self.bigram_frequencies = dict()
        self.total_bigrams = 0
        for sentence in sentences:
            prev_word = sentence[0]
            for word in sentence[1:]:
                self.bigram_frequencies[(prev_word, word)] = (
                    self.bigram_frequencies.get((prev_word, word), 0) + 1
                )
                self.total_bigrams += 1
                prev_word = word

        self.total_bigram_words = len(self.bigram_frequencies)

    def calculate_probability(self, prev_word, word):
        a = self.bigram_frequencies.get((prev_word, word), 0)
        b = self.unigram_frequencies.get(prev_word, 0)

        if b == 0:
            return 0
        return float(a) / float(b)

    def calculate_perplexity(self, prev_word, word):
        prob_word = self.calculate_probability(prev_word, word)
        if prob_word == 0:
            return 10 ** (7)
        perplexity = np.log2(float((1 / prob_word))) * (float(1 / self.total_bigrams))
        return perplexity

    def calculate_probability_smooth(self, prev_word, word, k, k_prev):
        a = self.bigram_frequencies.get((prev_word, word), 0)
        b = self.unigram_frequencies.get(prev_word, 0)

        return (float(a) + float(k)) / (float(b) + float(k_prev) + k * self.vocab_size)


class trigram_model(bigram_model):
    def __init__(self, sentences):
        bigram_model.__init__(self, sentences)
        self.trigram_frequencies = {}
        self.total_trigrams = 0
        for sentence in sentences:
            prev_word1 = sentence[0]
            prev_word2 = sentence[1]
            for word in sentence[2:]:
                self.trigram_frequencies[(prev_word1, prev_word2, word)] = (
                    self.trigram_frequencies.get((prev_word1, prev_word2, word), 0) + 1
                )
                prev_word1 = prev_word2
                prev_word2 = word
                self.total_trigrams += 1

        self.total_trigram_words = len(self.trigram_frequencies)

    def calculate_probability(self, prev_word1, prev_word2, word):
        trigram_frequency = self.trigram_frequencies.get(
            (prev_word1, prev_word2, word), 0
        )
        bigram_frequency = self.bigram_frequencies.get((prev_word2, word), 0)

        if bigram_frequency == 0:
            return 0
        return float(trigram_frequency) / float(bigram_frequency)

    def calculate_perplexity(self, prev_word1, prev_word2, word):
        prob_word = self.calculate_probability(prev_word1, prev_word2, word)
        if prob_word == 0:
            return 10 ** (7)
        perplexity = np.log2(float((1 / prob_word))) * (float(1 / self.total_trigrams))
        return perplexity

    def calculate_probability_smooth(self, prev_word1, prev_word2, word, k, k_prev):
        trigram_frequency = self.trigram_frequencies.get(
            (prev_word1, prev_word2, word), 0
        )
        bigram_frequency = self.bigram_frequencies.get((prev_word2, word), 0)

        return (float(trigram_frequency) + float(k)) / (
            float(bigram_frequency) + float(k_prev) + k * self.vocab_size
        )


class quadgram_model(trigram_model):
    def __init__(self, sentences):
        trigram_model.__init__(self, sentences)
        self.quadgram_frequencies = {}
        self.total_quadgrams = 0
        for sentence in sentences:
            prev_word1 = sentence[0]
            prev_word2 = sentence[1]
            prev_word3 = sentence[2]
            for word in sentence[3:]:
                quadgram = (prev_word1, prev_word2, prev_word3, word)
                self.quadgram_frequencies[quadgram] = (
                    self.quadgram_frequencies.get(quadgram, 0) + 1
                )
                prev_word1 = prev_word2
                prev_word2 = prev_word3
                prev_word3 = word
                self.total_quadgrams += 1

        self.total_quadgram_words = len(self.quadgram_frequencies)

    def calculate_probability(self, prev_word1, prev_word2, prev_word3, word):
        quadgram_frequency = self.quadgram_frequencies.get(
            (prev_word1, prev_word2, prev_word3, word), 0
        )
        trigram_frequency = self.trigram_frequencies.get(
            (prev_word1, prev_word2, prev_word3), 0
        )

        if trigram_frequency == 0:
            return 0
        return float(quadgram_frequency) / float(trigram_frequency)

    def calculate_perplexity(self, prev_word1, prev_word2, prev_word3, word):
        prob_word = self.calculate_probability(prev_word1, prev_word2, prev_word3, word)
        if prob_word == 0:
            return 10 ** (7)
        perplexity = np.log2(float((1 / prob_word))) * (float(1 / self.total_quadgrams))
        return perplexity

    def calculate_probability_smooth(
        self, prev_word1, prev_word2, prev_word3, word, k, k_prev
    ):
        quadgram_frequency = self.quadgram_frequencies.get(
            (prev_word1, prev_word2, prev_word3, word), 0
        )
        trigram_frequency = self.trigram_frequencies.get(
            (prev_word1, prev_word2, prev_word3), 0
        )

        return (float(quadgram_frequency) + float(k)) / (
            float(trigram_frequency) + float(k_prev) + k * self.vocab_size
        )

In [47]:
UNIGRAM_MODEL=unigram_model(train_unigram)

count=0
total_perplex = 0
for data in test_unigram:

    count+=1
    n = len(data)
    if(n == 0): continue
    unigram_perplex=0
    for word in data:
        unigram_perplex += np.log2(UNIGRAM_MODEL.calculate_probability(word))

    unigram_perplex *= (-1/n)
    final_perplex = 2 ** unigram_perplex
    total_perplex += final_perplex


avg_perplex = total_perplex/count

/tmp/ipykernel_8456/1632651415.py:12: RuntimeWarning: divide by zero encountered in log2
  unigram_perplex += np.log2(UNIGRAM_MODEL.calculate_probability(word))


In [48]:
avg_perplex

inf

In [50]:
BIGRAM_MODEL=bigram_model(train_bigram)
total_perplex=0
count=0
for data in test_bigram:
    count+= 1
    bigram_perplex = 0
    n = len(data)
    if(n < 2): continue

    for i in range(len(data)-1):
        bigram_perplex+=np.log2(BIGRAM_MODEL.calculate_probability(data[i],data[i+1]))

    bigram_perplex *= (-1/n)
    final_perplex = 2 ** bigram_perplex
    total_perplex += final_perplex


Avg_perplex=total_perplex/count
Avg_perplex

/tmp/ipykernel_8456/2328182941.py:11: RuntimeWarning: divide by zero encountered in log2
  bigram_perplex+=np.log2(BIGRAM_MODEL.calculate_probability(data[i],data[i+1]))


inf

In [51]:
TRIGRAM_MODEL=trigram_model(train_trigram)
total_perplex=0
count=0

for data in test_trigram:
    count+= 1
    trigram_perplex = 0
    n = len(data)
    if(n < 3): continue

    for i in range(len(data)-2):
        trigram_perplex+=np.log2(TRIGRAM_MODEL.calculate_probability(data[i],data[i+1], data[i+2]))

    trigram_perplex *= (-1/n)
    final_perplex = 2 ** trigram_perplex
    total_perplex += final_perplex


Avg_perplex=total_perplex/count
Avg_perplex

/tmp/ipykernel_8456/3141926806.py:12: RuntimeWarning: divide by zero encountered in log2
  trigram_perplex+=np.log2(TRIGRAM_MODEL.calculate_probability(data[i],data[i+1], data[i+2]))


inf

In [52]:
QUADGRAM_MODEL=quadgram_model(train_quadgram)
total_perplex=0
count=0

for data in test_quadgram:
    count+= 1
    quadgram_perplex = 0
    n = len(data)
    if(n < 4): continue

    for i in range(len(data)-3):
        quadgram_perplex+=np.log2(QUADGRAM_MODEL.calculate_probability(data[i],data[i+1], data[i+2] , data[i+3]))

    quadgram_perplex *= (-1/n)
    final_perplex = 2 ** quadgram_perplex
    total_perplex += final_perplex


Avg_perplex=total_perplex/count
Avg_perplex

/tmp/ipykernel_8456/3850654556.py:12: RuntimeWarning: divide by zero encountered in log2
  quadgram_perplex+=np.log2(QUADGRAM_MODEL.calculate_probability(data[i],data[i+1], data[i+2] , data[i+3]))


inf

In [23]:
UNIGRAM_MODEL=unigram_model(train_unigram)

for data in test_unigram:
    for word in data:
        if word not in UNIGRAM_MODEL.vocabulary:
            UNIGRAM_MODEL.vocabulary.add(word)
            UNIGRAM_MODEL.vocab_size += 1
            UNIGRAM_MODEL.unigram_frequencies[word] = 0

In [24]:
def uni(k):

    count=0
    total_perplex = 0
    for data_ in test_unigram:

        count+=1
        n = len(data_)
        if(n == 0): continue
        unigram_perplex=0
        for word in data_:
            unigram_perplex += np.log2(UNIGRAM_MODEL.calculate_probability_smooth(word , k))

        unigram_perplex *= (-1/float(n))
        final_perplex = 2 ** unigram_perplex
        total_perplex += final_perplex

    avg_perplex = total_perplex/float(count)
    return avg_perplex

In [25]:
from scipy.optimize import minimize

In [26]:
def callback(xk):
    print(xk)

In [27]:
result = minimize(uni , 0.001 , tol = 1e-5 , callback = callback, bounds = [(0.0001 , 10)])


[0.37064447]
[0.44503719]
[0.63115028]
[0.84244655]
[1.15868659]
[1.58415816]
[2.17805797]
[2.98456507]
[4.06187246]
[5.44407332]
[7.11149037]
[8.92991313]
[10.]


In [32]:
k_uni = 10
uni(k_uni)

8213.009889641138

In [29]:
BIGRAM_MODEL=bigram_model(train_bigram)

for data in test_bigram:
    n = len(data)
    for word in data:
        if word not in BIGRAM_MODEL.vocabulary:
            BIGRAM_MODEL.vocabulary.add(word)
            BIGRAM_MODEL.vocab_size += 1

    for i in range(n-1):

        if(data[i], data[i+1]) not in BIGRAM_MODEL.bigram_frequencies:
            BIGRAM_MODEL.bigram_frequencies[(data[i], data[i+1])] = 0

In [33]:

def bi(k):

    total_perplex=0
    count=0

    for data in test_bigram:
        count+= 1
        bigram_perplex = 0
        n = len(data)
        if(n < 2): continue

        for i in range(len(data)-1):

            bigram_perplex+=np.log2(BIGRAM_MODEL.calculate_probability_smooth(data[i],data[i+1] , k ,k_uni ))

        bigram_perplex *= (-1/n)
        final_perplex = 2 ** bigram_perplex
        total_perplex += final_perplex


    Avg_perplex=total_perplex/count
    return Avg_perplex


In [34]:
result = minimize(bi , 1e-8 , tol = 1e-5 , callback = callback , bounds = [(1e-8 , 1)])

[1.67172134e-05]
[1.84732615e-05]
[2.80747062e-05]
[3.7494697e-05]
[5.25929876e-05]
[7.24219734e-05]
[0.0001005]
[0.00013913]
[0.00019293]
[0.00026758]
[0.00037141]
[0.00051574]
[0.00071624]
[0.00099409]
[0.0013774]
[0.00190211]
[0.00261162]
[0.00355294]
[0.00476611]
[0.00626179]
[0.00798341]
[0.00976209]
[0.01130675]
[0.01231719]
[0.01273346]
[0.01281533]


In [35]:
k_bi = 0.01303768
bi(k_bi)


1825.346134736325

In [43]:
TRIGRAM_MODEL=trigram_model(train_trigram)

y = []
for data in test_trigram:
    n = len(data)
    for word in data:
        if word not in TRIGRAM_MODEL.vocabulary:
            TRIGRAM_MODEL.vocabulary.add(word)
            TRIGRAM_MODEL.vocab_size += 1

    for i in range(n-2):
        if(data[i], data[i+1], data[i+2]) not in TRIGRAM_MODEL.trigram_frequencies:
            TRIGRAM_MODEL.trigram_frequencies[(data[i], data[i+1] , data[i+2])] = 0

In [44]:
def tri(k):

    total_perplex=0
    count=0

    for data in test_trigram:
        count+= 1
        trigram_perplex = 0
        n = len(data)
        if(n < 3): continue

        for i in range(len(data)-2):
            trigram_perplex+=np.log2(TRIGRAM_MODEL.calculate_probability_smooth(data[i],data[i+1], data[i+2] ,k , k_bi))

        trigram_perplex *= (-1/n)
        final_perplex = 2 ** trigram_perplex
        total_perplex += final_perplex


    Avg_perplex=total_perplex/count
    return Avg_perplex

In [45]:
result = minimize(tri , 0.001 , tol = 1e-5 , callback = callback , bounds = [(0.0001 , 1)])

[0.00115661]
[0.00114889]


In [46]:
k_tri = 0.00114889
tri(k_tri)


2334.3734379988905

In [47]:
QUADGRAM_MODEL= quadgram_model(train_quadgram)

y = []
for data in test_quadgram:
    n = len(data)
    for word in data:
        if word not in QUADGRAM_MODEL.vocabulary:
            QUADGRAM_MODEL.vocabulary.add(word)
            QUADGRAM_MODEL.vocab_size += 1

    for i in range(n-3):
        if(data[i], data[i+1], data[i+2] , data[i+3]) not in QUADGRAM_MODEL.quadgram_frequencies:
            QUADGRAM_MODEL.quadgram_frequencies[(data[i], data[i+1] , data[i+2] , data[i+3])] = 0

In [50]:
def quad(k):

    total_perplex=0
    count=0

    for data in test_quadgram:
        count+= 1
        quadgram_perplex = 0
        n = len(data)
        if(n < 4): continue

        for i in range(len(data)-3):
            quadgram_perplex += np.log2(QUADGRAM_MODEL.calculate_probability_smooth(data[i],data[i+1], data[i+2],data[i+3] ,k, k_tri))

        quadgram_perplex *= (-1/n)
        final_perplex = 2 ** quadgram_perplex
        total_perplex += final_perplex


    Avg_perplex=total_perplex/count
    return Avg_perplex

In [51]:
result = minimize(quad , 0.001 , tol = 1e-5 , callback = callback , bounds = [(0.00001 , 1)])

[0.00064617]
[0.00043052]
[0.00040645]
[0.00040418]


In [52]:
quad(0.00040418)

1882.1098191057736

In [ ]:
class N_Gram(unigram_model):
    def __init__(self,sentences,n):
        if(n==1):
            self.model=unigram_model(sentences)
        if(n==2):
            self.model=bigram_model(sentences)
        if(n==3):
            self.model=trigram_model(sentences)
        if(n==4):
            self.model=quadgram_model(sentences)

In [ ]:
BIGRAM_MODEL.bigram_frequencies

{('<s>', '<s>'): 1345530,
 ('<s>', 'how'): 1916,
 ('how', 'about'): 177,
 ('about', 'you'): 185,
 ('you', 'let'): 19,
 ('let', 'people'): 80,
 ('people', 'believe'): 57,
 ('believe', 'what'): 71,
 ('what', 'they'): 755,
 ('they', 'want'): 600,
 ('want', 'to'): 2618,
 ('to', 'believe'): 437,
 ('believe', 'in'): 539,
 ('in', 'a'): 3161,
 ('a', 'hard'): 110,
 ('hard', 'time'): 61,
 ('time', 'like'): 15,
 ('like', 'this'): 1091,
 ('this', '</s>'): 2322,
 ('</s>', '</s>'): 1345530,
 ('<s>', 'stop'): 309,
 ('stop', 'causing'): 5,
 ('causing', 'more'): 11,
 ('more', 'division'): 8,
 ('division', 'when'): 5,
 ('when', 'the'): 804,
 ('the', 'world'): 2614,
 ('world', 'is'): 240,
 ('is', 'already'): 170,
 ('already', 'divided'): 6,
 ('divided', '</s>'): 15,
 ('<s>', 'seems'): 176,
 ('seems', 'to'): 383,
 ('to', 'me'): 762,
 ('me', 'that'): 223,
 ('that', 'you'): 1657,
 ('you', 'are'): 3971,
 ('are', 'the'): 1337,
 ('the', 'naive'): 5,
 ('naive', 'one'): 2,
 ('one', '</s>'): 667,
 ('<s>', 'you'):

### Using Good Turing Smoothing

In [43]:
UNIGRAM_MODEL=unigram_model(train_unigram)

count=0
total_perplex = 0
UNIGRAM_MODEL.calculate_new_word_count(UNIGRAM_MODEL.unigram_frequencies)
for data in test_unigram:

    count+=1
    n = len(data)
    if(n == 0): continue
    unigram_perplex=0
    for word in data:
        unigram_perplex += np.log2(UNIGRAM_MODEL.calculate_probability_good_smooth(word))

    unigram_perplex *= (-1/n)
    final_perplex = 2 ** unigram_perplex
    total_perplex += final_perplex


avg_perplex = total_perplex/count
avg_perplex

11066.516122294264

In [44]:
BIGRAM_MODEL=bigram_model(train_bigram)
total_perplex=0
count=0
BIGRAM_MODEL.calculate_new_word_count(BIGRAM_MODEL.bigram_frequencies)
for data in test_bigram:
    count+= 1
    bigram_perplex = 0
    n = len(data)
    if(n < 2): continue

    for i in range(len(data)-1):
        bigram_perplex+=np.log2(BIGRAM_MODEL.calculate_probability_good_smooth((data[i],data[i+1])))

    bigram_perplex *= (-1/n)
    final_perplex = 2 ** bigram_perplex
    total_perplex += final_perplex


Avg_perplex=total_perplex/count
Avg_perplex

33521.19878052262

In [45]:
TRIGRAM_MODEL=trigram_model(train_trigram)
total_perplex=0
count=0
TRIGRAM_MODEL.calculate_new_word_count(TRIGRAM_MODEL.trigram_frequencies)

for data in test_trigram:
    count+= 1
    trigram_perplex = 0
    n = len(data)
    if(n < 3): continue

    for i in range(len(data)-2):
        trigram_perplex+=np.log2(TRIGRAM_MODEL.calculate_probability_good_smooth((data[i],data[i+1], data[i+2])))

    trigram_perplex *= (-1/n)
    final_perplex = 2 ** trigram_perplex
    total_perplex += final_perplex


Avg_perplex=total_perplex/count
Avg_perplex

49041.468127868095

In [46]:
QUADGRAM_MODEL=quadgram_model(train_quadgram)
total_perplex=0
count=0
QUADGRAM_MODEL.calculate_new_word_count(QUADGRAM_MODEL.quadgram_frequencies)

for data in test_quadgram:
    count+= 1
    quadgram_perplex = 0
    n = len(data)
    if(n < 4): continue

    for i in range(len(data)-3):
        quadgram_perplex+=np.log2(QUADGRAM_MODEL.calculate_probability_good_smooth((data[i],data[i+1], data[i+2] , data[i+3])))

    quadgram_perplex *= (-1/n)
    final_perplex = 2 ** quadgram_perplex
    total_perplex += final_perplex


Avg_perplex=total_perplex/count
Avg_perplex

38494.41832202575